In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import urllib.request
import requests


In [126]:

# extract pages 

# this is input url
html_page = requests.get('https://www.cnbc.com/hospitals/').text

soup = BeautifulSoup(html_page, "lxml")
for link in soup.findAll('a'):
    links_list.append(link.get('href'))

#sel_links_list = [i for i in links_list if (("html" in str(i) or "page" in str(i)) and ("video" not in str(i)))]
last_page = [i for i in links_list if ("?page" in str(i))]
last_page_num = int(re.findall(r'\d+',last_page[len(last_page)-1])[0])
last_page_num
 

21

In [1]:

# extract all pages linked to url
# extract all articles in each page of url
# saves article, url, author, headline, posttime in json for each article

for page_num in range(1,last_page_num+1):
    
    url_path = 'https://www.cnbc.com/hospitals/'+ '?page' + str(page_num)
    links_list = []
    html_page = requests.get('https://www.cnbc.com/hospitals/').text
    soup = BeautifulSoup(html_page, "lxml")
    for link in soup.findAll('a'):
        links_list.append(link.get('href'))

    sel_links_list = [i for i in links_list if (("html" in str(i)) and ("video" not in str(i)))]
    for link_num in range(len(sel_links_list)):
        article_url = 'https://www.cnbc.com' + str(sel_links_list[link_num])
        page = urllib.request.urlopen(article_url)
        soup = BeautifulSoup(page, 'html.parser')
        
        headline_soup = soup.findAll('h1', attrs={'class': 'ArticleHeader-headline'})
        headline_soup = re.sub('<.*?>', '', str(headline_soup))
        headline = str.replace(headline_soup, "\n","")

        author_soup = soup.findAll('div', attrs={'class': 'Author-authorNameAndSocial'})
        author_soup = re.sub('<.*?>', '', str(author_soup))
        author = str.replace(author_soup, "\n","")

        time_soup = soup.findAll('div', attrs={'class': 'ArticleHeader-time'})
        time_soup = re.sub('<.*?>', '', str(time_soup))
        time = str.replace(time_soup, "\n","")
        
        
        text_soup = soup.findAll('div', attrs={'class': 'group'})
        text_soup = re.sub('<.*?>', '', str(text_soup))
        text = str.replace(text_soup, "\n","")
        
        save_dataframe = pd.DataFrame({'article_url' : article_url,
                      'headline' : headline,
                      'author' : author,
                      'time' : time,
                      'article' : text},index=[0])                                        
                                                  
        save_path = "C:/Users/ak18474/Jupyter notebooks/Web scraping/" + "page_" + str(page_num)+ "_file_" + str(link_num) + ".json"
        save_dataframe.to_json(save_path)
        